In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pytorch-metric-learning


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.1/119.1 kB 2.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [3]:
import pandas as pd
import os
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from pytorch_metric_learning import distances, losses, miners, reducers, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator

folder = '/content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES'
all_pictures = os.listdir(folder)

file_path = "/content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES.csv"
df = pd.read_csv(file_path)

# Reemplazar el prefijo "c." con nada en la columna 'date'
#df['date'] = df['date'].str.replace('c.', '').astype('int', errors='ignore')


print(df.head())

NUM_CLASSES = len(df['date'].unique().tolist())

print("Number of classes:", NUM_CLASSES)


                  artist  date      genre  pixelsx  pixelsy  size_bytes  \
0  Ernst Ludwig Kirchner  1913  landscape   4908.0   3684.0  10682834.0   
1          Koloman Moser  1913  landscape   4716.0   3420.0  10933062.0   
2          Koloman Moser  1913  landscape   4722.0   3102.0   9018058.0   
3       Ferdinand Hodler  1913   portrait   3156.0   4638.0   8453089.0   
4          Koloman Moser  1913     design   4734.0   2622.0   8317658.0   

    source                 style  \
0  wikiart         Expressionism   
1  wikiart             Symbolism   
2  wikiart             Symbolism   
3  wikiart  Art Nouveau (Modern)   
4  wikiart  Art Nouveau (Modern)   

                                            title    artist_group  in_train  \
0                           Red Tree on the Beach  train_and_test      True   
1                                 Mountain ranges  train_and_test      True   
2                          Landscape in Semmering  train_and_test      True   
3      Valentine

In [4]:
import os
import shutil
import random
import pandas as pd


In [5]:
from PIL import Image
Image.MAX_IMAGE_PIXELS = None

In [6]:
# GROUP THE FILES BY EACH date
import random
def group_files_per_date():
  data_per_date = {}

  # Iterate over each row of the DataFrame
  for index, row in df.iterrows():
      date = row['date']
      new_filename = row['new_filename']

      # Check if the date is already in the dictionary
      if date in data_per_date:
          # Add the new_filename to the existing list of the date
          data_per_date[date].append(new_filename)
      else:
          # Create a new list for the date and add the new_filename
          data_per_date[date] = [new_filename]

  return data_per_date

# SPLIT THE FILES INTO TRAIN AND TEST SETS (CREATE NEW DATASETS)
def split_test_train(data_per_date, train_percentage = 0.8):

  TRAIN_names = []
  TEST_names = []

  # Iterate over each date and their 'new_filename'
  for date, filenames in data_per_date.items():
      # Calculate the number of files for training and testing
      total_files = len(filenames)
      num_train = int(train_percentage * total_files)

      # Shuffle the filenames to avoid selection biases
      random.shuffle(filenames)

      # Divide the filenames into train and test
      train_filenames = filenames[:num_train]
      test_filenames = filenames[num_train:]

      # Store the divided filenames into train and test
      TRAIN_names.extend(train_filenames)
      TEST_names.extend(test_filenames)
  return TRAIN_names, TEST_names



# CREATE TEST and TRAIN folders
def create_train_test_folders(folder, TRAIN_names, TEST_names):
    # Output folder for TRAIN and TEST
    train_folder = os.path.join(folder, 'TRAIN')
    test_folder = os.path.join(folder, 'TEST')

    # Create TRAIN and TEST folders if they don't exist
    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(test_folder, exist_ok=True)

    # Move images from TRAIN to TRAIN folder
    for filename in TRAIN_names:
        src = os.path.join(folder, filename)
        dst = os.path.join(train_folder, filename)
        shutil.move(src, dst)

    # Move images from TEST to TEST folder
    for filename in TEST_names:
        src = os.path.join(folder, filename)
        dst = os.path.join(test_folder, filename)
        shutil.move(src, dst)

    # Create df_train with rows whose 'new_filename' is in TRAIN_names and keep only 'date' and 'new_filename' columns
    train_dataframe = df[df['new_filename'].isin(TRAIN_names)][['date', 'new_filename']]
    # File path for the CSV file
    file_path = os.path.join(folder, 'train_dataframe.csv')
    # Save the DataFrame as CSV
    train_dataframe.to_csv(file_path, index=False)

    # Create df_test with rows whose 'new_filename' is in TEST_names and keep only 'date' and 'new_filename' columns
    test_dataframe = df[df['new_filename'].isin(TEST_names)][['date', 'new_filename']]
    # File path for the CSV file
    file_path = os.path.join(folder, 'test_dataframe.csv')
    # Save the DataFrame as CSV
    test_dataframe.to_csv(file_path, index=False)

    return train_dataframe, test_dataframe

if not (os.path.exists(os.path.join(folder, 'TRAIN')) and os.path.exists(os.path.join(folder, 'TEST')) and os.path.exists(os.path.join(folder, 'train_dataframe.csv'))):
  # Group files per date
  data_per_date = group_files_per_date()
  # Print the number of files for each date
  print("- date and numer of images:")
  for key in data_per_date.keys():
    print(key, len(data_per_date[key]))

  # Split data
  TRAIN_names, TEST_names = split_test_train(data_per_date)

  # Use the function to create TRAIN and TEST folders
  train_dataframe, test_dataframe = create_train_test_folders(folder, TRAIN_names, TEST_names)
  print("\nImages successfully moved to TRAIN and TEST folders.")
else:
  print("TRAIN and TEST folders already exist. Reading train and test DataFrames from CSV files...")

  # Read train and test DataFrames from CSV files
  train_folder = os.path.join(folder, 'TRAIN')
  test_folder = os.path.join(folder, 'TEST')

  train_images = set(os.listdir(train_folder))
  test_images = set(os.listdir(test_folder))

  train_dataframe = df[df['new_filename'].isin(train_images)].copy()
  test_dataframe = df[df['new_filename'].isin(test_images)].copy()

  train_dataframe = train_dataframe[['new_filename', 'date']]
  test_dataframe = test_dataframe[['new_filename', 'date']]

  print("Train DataFrame:", train_dataframe.shape)
  print("Test DataFrame:", test_dataframe.shape)


TRAIN and TEST folders already exist. Reading train and test DataFrames from CSV files...
Train DataFrame: (1200, 2)
Test DataFrame: (300, 2)


In [7]:
test_dataframe.groupby('date').size().reset_index(name='count')

,date,count
0,1885,20
1,1888,20
2,1889,20
3,1890,20
4,1895,20
5,1907,20
6,1908,20
7,1909,20
8,1910,20
9,1911,20


In [65]:
num_valores_distintos = test_dataframe['date'].nunique()
print("Número de valores distintos en la columna 'date':", num_valores_distintos)


Número de valores distintos en la columna 'date': 15


# Model

In [94]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader

class ArtEmbeddingModel(nn.Module):
    def __init__(self):
        super(ArtEmbeddingModel, self).__init__()
        # Load pre-trained DenseNet model
        self.base_model = models.densenet121(pretrained=True)
        # Modify the classifier to output embeddings of size 128
        num_features = self.base_model.classifier.in_features
        self.base_model.classifier = nn.Linear(num_features, 128)

    def forward(self, x):
        # Forward pass through the base model
        features = self.base_model(x)
        return features

def triplet_loss(anchor, positive, negative, margin=1.0):
    distance_positive = (anchor - positive).pow(2).sum(1)  # Distancia L2
    distance_negative = (anchor - negative).pow(2).sum(1)  # Distancia L2
    losses = torch.relu(distance_positive - distance_negative + margin)
    return losses.mean()




# Triplets creation functions

In [54]:
# Definir la clase TripletArtDataset
class TripletArtDataset(Dataset):
    def __init__(self, images, dates, similarity_threshold=5, transform=None):
        self.images = images
        self.dates = dates
        self.similarity_threshold = similarity_threshold
        self.transform = transform
        self.triplets = self.create_triplets()

    def create_triplets(self):
        triplets = []
        num_images = len(self.images)

        for i in range(num_images):
            anchor_image = self.images[i]
            anchor_date = self.dates[i]

            positive_indices = [j for j in range(num_images) if abs(self.dates[j] - anchor_date) <= self.similarity_threshold and j != i]
            negative_indices = [j for j in range(num_images) if abs(self.dates[j] - anchor_date) > self.similarity_threshold]

            if not positive_indices or not negative_indices:
                continue

            positive_index = np.random.choice(positive_indices)
            negative_index = np.random.choice(negative_indices)

            positive_image = self.images[positive_index]
            negative_image = self.images[negative_index]

            triplets.append((anchor_image, positive_image, negative_image))

        return triplets

    def __getitem__(self, index):
        anchor_path, positive_path, negative_path = self.triplets[index]

        # Imprimir las rutas para depuración
        print(f"Anchor path: {anchor_path}")
        print(f"Positive path: {positive_path}")
        print(f"Negative path: {negative_path}")

        try:
            anchor = Image.open(anchor_path)
            positive = Image.open(positive_path)
            negative = Image.open(negative_path)
        except FileNotFoundError as e:
            print(e)
            return None, None, None

        if self.transform:
            anchor = self.transform(anchor)
            positive = self.transform(positive)
            negative = self.transform(negative)

        return anchor, positive, negative

    def __len__(self):
        return len(self.triplets)

# Train and Test Dataloaders

In [51]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np


# Asegúrate de que las rutas de las imágenes son completas
train_images = set(os.listdir(train_folder))
train_dataframe = df[df['new_filename'].isin(train_images)].copy()
train_dataframe = train_dataframe[['new_filename', 'date']]

# Crear una lista con las rutas completas de las imágenes
train_images_list = [os.path.join(train_folder, img) for img in train_images]

dates_train = []
for image_name in train_images:
    matching_row = train_dataframe[train_dataframe['new_filename'] == image_name]
    if not matching_row.empty:
        # Agregar la fecha correspondiente a la lista
        dates_train.append(matching_row.iloc[0]['date'])  # Asumimos que solo hay una fila que coincida
    else:
        # Manejo de casos donde no se encuentra la imagen
        dates_train.append(None)  # O cualquier otro valor indicativo de ausencia

# Convertir la lista filtrada de fechas a enteros
dates_train_int = [int(date) for date in dates_train]

print(dates_train_int)

# Definir las transformaciones
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])



# Crear datasets y dataloaders para entrenamiento
train_dataset = TripletArtDataset(train_images_list, dates_train_int, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)





[1912, 1888, 1912, 1917, 1914, 1895, 1895, 1908, 1917, 1885, 1895, 1917, 1889, 1885, 1915, 1885, 1885, 1909, 1912, 1888, 1914, 1913, 1895, 1910, 1907, 1888, 1885, 1908, 1895, 1913, 1907, 1917, 1908, 1917, 1885, 1889, 1888, 1909, 1888, 1915, 1885, 1911, 1909, 1888, 1914, 1912, 1911, 1888, 1895, 1908, 1885, 1917, 1911, 1895, 1888, 1895, 1908, 1885, 1907, 1890, 1908, 1889, 1917, 1890, 1890, 1917, 1908, 1913, 1914, 1888, 1913, 1889, 1910, 1888, 1907, 1910, 1913, 1909, 1910, 1909, 1907, 1907, 1917, 1907, 1911, 1910, 1895, 1909, 1910, 1915, 1910, 1912, 1907, 1912, 1909, 1895, 1908, 1910, 1911, 1888, 1885, 1885, 1917, 1910, 1910, 1910, 1888, 1910, 1914, 1889, 1915, 1910, 1908, 1888, 1909, 1890, 1914, 1907, 1911, 1889, 1915, 1913, 1912, 1890, 1907, 1914, 1912, 1914, 1885, 1888, 1911, 1915, 1910, 1908, 1907, 1913, 1915, 1895, 1908, 1911, 1910, 1895, 1885, 1914, 1890, 1913, 1910, 1890, 1908, 1913, 1908, 1915, 1913, 1913, 1888, 1885, 1914, 1885, 1889, 1889, 1885, 1908, 1913, 1917, 1895, 1913, 191

In [56]:
# Asegúrate de que las rutas de las imágenes son completas
test_images = set(os.listdir(test_folder))
test_dataframe = df[df['new_filename'].isin(test_images)].copy()
test_dataframe = test_dataframe[['new_filename', 'date']]

# Crear una lista con las rutas completas de las imágenes
test_images_list = [os.path.join(test_folder, img) for img in test_images]

dates_test = []
for image_name in test_images:
    matching_row = test_dataframe[test_dataframe['new_filename'] == image_name]
    if not matching_row.empty:
        # Agregar la fecha correspondiente a la lista
        dates_test.append(matching_row.iloc[0]['date'])  # Asumimos que solo hay una fila que coincida
    else:
        # Manejo de casos donde no se encuentra la imagen
        dates_test.append(None)  # O cualquier otro valor indicativo de ausencia

# Convertir la lista filtrada de fechas a enteros
dates_test_int = [int(date) for date in dates_test]

print(dates_test_int)

# Definir las transformaciones
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])



# Crear datasets y dataloaders para entrenamiento
test_dataset = TripletArtDataset(test_images_list, dates_test_int, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

[1917, 1910, 1888, 1907, 1888, 1912, 1889, 1914, 1909, 1910, 1885, 1890, 1913, 1885, 1907, 1911, 1912, 1908, 1915, 1913, 1888, 1909, 1889, 1895, 1910, 1888, 1913, 1911, 1895, 1914, 1912, 1917, 1912, 1909, 1910, 1910, 1910, 1907, 1885, 1909, 1915, 1908, 1885, 1915, 1917, 1912, 1895, 1912, 1911, 1890, 1889, 1907, 1915, 1889, 1889, 1913, 1914, 1915, 1914, 1895, 1885, 1889, 1917, 1890, 1914, 1917, 1908, 1885, 1890, 1917, 1915, 1912, 1895, 1914, 1912, 1917, 1889, 1907, 1889, 1911, 1911, 1913, 1895, 1908, 1885, 1885, 1915, 1907, 1889, 1915, 1911, 1911, 1890, 1895, 1888, 1890, 1917, 1889, 1889, 1890, 1907, 1909, 1914, 1895, 1915, 1889, 1911, 1914, 1889, 1890, 1915, 1888, 1908, 1917, 1917, 1909, 1913, 1910, 1885, 1911, 1908, 1885, 1917, 1908, 1915, 1912, 1912, 1910, 1915, 1908, 1909, 1914, 1890, 1908, 1895, 1911, 1890, 1912, 1885, 1910, 1912, 1895, 1888, 1888, 1917, 1888, 1911, 1890, 1909, 1917, 1909, 1912, 1914, 1907, 1910, 1907, 1911, 1907, 1890, 1889, 1907, 1910, 1911, 1908, 1907, 1908, 191

# ENTRENAMENT

In [99]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ArtEmbeddingModel().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)
num_epochs = 25

for epoch in range(num_epochs):
    model.train()
    for anchors, positives, negatives in train_dataloader:
        anchors = anchors.to(device)
        positives = positives.to(device)
        negatives = negatives.to(device)

        # Imprimir las rutas para depuración
        print(f"Anchor path: {anchors}")
        print(f"Positive path: {positives}")
        print(f"Negative path: {negatives}")

        optimizer.zero_grad()
        anchor_embeddings = model(anchors)
        positive_embeddings = model(positives)
        negative_embeddings = model(negatives)

        loss = triplet_loss(anchor_embeddings, positive_embeddings, negative_embeddings)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Se han truncado las últimas 5000 líneas del flujo de salida.


        ...,


        [[[0.6667, 0.7961, 0.8275,  ..., 0.8000, 0.8510, 0.8784],
          [0.8039, 0.8000, 0.6980,  ..., 0.6824, 0.8784, 0.8392],
          [0.7608, 0.8235, 0.7725,  ..., 0.6392, 0.8667, 0.8078],
          ...,
          [0.8118, 0.8000, 0.7922,  ..., 0.8157, 0.8196, 0.8157],
          [0.8157, 0.7961, 0.7804,  ..., 0.8392, 0.8353, 0.8275],
          [0.7922, 0.7725, 0.7569,  ..., 0.8627, 0.8510, 0.8392]],

         [[0.6392, 0.7686, 0.7922,  ..., 0.7725, 0.8118, 0.8353],
          [0.7725, 0.7686, 0.6588,  ..., 0.6549, 0.8392, 0.7961],
          [0.7294, 0.7922, 0.7412,  ..., 0.6078, 0.8353, 0.7725],
          ...,
          [0.7569, 0.7451, 0.7373,  ..., 0.7490, 0.7451, 0.7412],
          [0.7725, 0.7529, 0.7373,  ..., 0.7647, 0.7529, 0.7451],
          [0.7569, 0.7373, 0.7137,  ..., 0.7882, 0.7686, 0.7569]],

         [[0.5765, 0.7059, 0.7333,  ..., 0.7020, 0.7059, 0.7098],
          [0.7216, 0.7176, 0.6

# EVALUATE MODEL

In [100]:
def calculate_embeddings(model, dataloader, device):
    model.eval()
    embeddings = []
    labels = []

    with torch.no_grad():
        for anchors, positives, negatives in dataloader:
            anchors = anchors.to(device)
            embeddings.append(model(anchors).cpu())
            labels.append(anchors)

    embeddings = torch.cat(embeddings)
    return embeddings, labels


In [101]:
train_embeddings, train_labels = calculate_embeddings(model, train_dataloader, device)
test_embeddings, test_labels = calculate_embeddings(model, test_dataloader, device)


Anchor path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TRAIN/100830.jpg
Positive path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TRAIN/81346.jpg
Negative path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TRAIN/98413.jpg
Anchor path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TRAIN/62516.jpg
Positive path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TRAIN/25799.jpg
Negative path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TRAIN/12301.jpg
Anchor path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TRAIN/25086.jpg
Positive path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TRAIN/35207.jpg
Negative path: /content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TRAIN/10307.jpg
Anchor path: /content/drive/My Dr

In [102]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Convertir las fechas a categorías para la clasificación
train_labels = np.array(dates_train_int)
test_labels = np.array(dates_test_int)

# Ajustar un clasificador k-NN en los embeddings del conjunto de entrenamiento
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(train_embeddings, train_labels)

# Predecir las etiquetas del conjunto de prueba
test_predictions = knn.predict(test_embeddings)

# Calcular la precisión
accuracy = np.mean(test_predictions == test_labels)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 9.33%


In [ ]:
'''
import os
from PIL import Image
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from pytorch_metric_learning import distances, losses, miners, reducers, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator

# Define the transformation to convert images to PyTorch tensors
transf = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4902860351171657, 0.4327027123349515, 0.3679902021355657], std=[0.22134062718873712, 0.20430693703606892, 0.18851782142531143])  # Normalization for color images
])
# Paths of the training and testing folders
train_folder = '/content/drive/My Drive/PROJECTE DEEP LEARNING/TOP5_ARTISTS_WITH_100_PICTURES/TRAIN'
test_folder = '/content/drive/My Drive/PROJECTE DEEP LEARNING/TOP5_ARTISTS_WITH_100_PICTURES/TEST'

# Get the file names of training and testing images from the dataframes
train_filenames = train_dataframe['new_filename'].tolist()
test_filenames = test_dataframe['new_filename'].tolist()
# Create empty lists for images and labels
train_images = []
train_labels = []
test_images = []
test_labels = []
#if not (os.path.exists(os.path.join(folder, 'TRAIN')) and os.path.exists(os.path.join(folder, 'TEST'))):

  # Load training images
for filename in train_filenames:
    img_path = os.path.join(train_folder, filename)
    img = Image.open(img_path)
    img_tensor = transf(img)
    train_images.append(img_tensor)
    # Get label from the dataframe
    label = train_dataframe[train_dataframe['new_filename'] == filename]['date'].iloc[0]
    train_labels.append(label)

# Load testing images
for filename in test_filenames:
    img_path = os.path.join(test_folder, filename)
    img = Image.open(img_path)
    img_tensor = transf(img)
    test_images.append(img_tensor)
    # Get label from the dataframe
    label = test_dataframe[test_dataframe['new_filename'] == filename]['date'].iloc[0]
    test_labels.append(label)

# Convert labels to tensor
train_labels_tensor = torch.tensor(train_labels, dtype=torch.float32)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.float32)

# Create PyTorch datasets
train_dataset = TensorDataset(torch.stack(train_images), train_labels_tensor)
test_dataset = TensorDataset(torch.stack(test_images), test_labels_tensor)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=50, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=50, shuffle=False)
'''

"\nimport os\nfrom PIL import Image\nimport torch\nfrom torchvision import transforms\nfrom torch.utils.data import DataLoader, TensorDataset\nimport pandas as pd\nimport torch.nn as nn\nimport torch.optim as optim\nimport torch.nn.functional as F\nfrom pytorch_metric_learning import distances, losses, miners, reducers, testers\nfrom pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator\n\n# Define the transformation to convert images to PyTorch tensors\ntransf = transforms.Compose([\n    transforms.ToTensor(),\n    transforms.Normalize(mean=[0.4902860351171657, 0.4327027123349515, 0.3679902021355657], std=[0.22134062718873712, 0.20430693703606892, 0.18851782142531143])  # Normalization for color images\n])\n# Paths of the training and testing folders\ntrain_folder = '/content/drive/My Drive/PROJECTE DEEP LEARNING/TOP5_ARTISTS_WITH_100_PICTURES/TRAIN'\ntest_folder = '/content/drive/My Drive/PROJECTE DEEP LEARNING/TOP5_ARTISTS_WITH_100_PICTURES/TEST'\n\n# Get the f

In [ ]:
'''
import matplotlib.pyplot as plt
import numpy as np

def show_images(dataset, labels, class_name, num_images=4):
    # Find indices of images belonging to the specified class
    class_indices = [idx for idx, label in enumerate(labels) if label == class_name]
    # Randomly select 4 indices
    selected_indices = np.random.choice(class_indices, num_images)

    # Plot the images
    fig, axes = plt.subplots(1, num_images, figsize=(15, 3))
    fig.suptitle(f"Class {class_name} Images")
    for i, idx in enumerate(selected_indices):
        img, label = dataset[idx]
        img = img.permute(1, 2, 0)  # Convert from (C, H, W) to (H, W, C) for visualization
        axes[i].imshow(img)
        axes[i].axis('off')
    plt.show()

# Example: Show 4 images of class 0 from the training set
show_images(train_dataset, train_labels_tensor, class_name=1913)

# Example: Show 4 images of class 1 from the testing set
show_images(test_dataset, test_labels_tensor, class_name=1913)
'''

'\nimport matplotlib.pyplot as plt\nimport numpy as np\n\ndef show_images(dataset, labels, class_name, num_images=4):\n    # Find indices of images belonging to the specified class\n    class_indices = [idx for idx, label in enumerate(labels) if label == class_name]\n    # Randomly select 4 indices\n    selected_indices = np.random.choice(class_indices, num_images)\n\n    # Plot the images\n    fig, axes = plt.subplots(1, num_images, figsize=(15, 3))\n    fig.suptitle(f"Class {class_name} Images")\n    for i, idx in enumerate(selected_indices):\n        img, label = dataset[idx]\n        img = img.permute(1, 2, 0)  # Convert from (C, H, W) to (H, W, C) for visualization\n        axes[i].imshow(img)\n        axes[i].axis(\'off\')\n    plt.show()\n\n# Example: Show 4 images of class 0 from the training set\nshow_images(train_dataset, train_labels_tensor, class_name=1913)\n\n# Example: Show 4 images of class 1 from the testing set\nshow_images(test_dataset, test_labels_tensor, class_name=

In [ ]:
import os
from PIL import Image
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from pytorch_metric_learning import distances, losses, miners, reducers, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator



# NETWORK

In [ ]:
from pytorch_metric_learning import losses

In [ ]:
### MNIST code originally from https://github.com/pytorch/examples/blob/master/mnist/main.py ###
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(64 * 110 * 110, 128)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        return x